# Industrialized Analytics & MLOps  
## Data Preparation in SageMaker Notebooks with SQL using AWS Athena. ML Using Comprehend NLP
`(Revision History:
PA5, 2019-10-!9, @akirmak: Comprehend NLP added for sentiment analysis
PA4, 2019-10-!9, @akirmak: Scalable data prep using Glue, Athena & SageMaker
`

## Big Data & AI/ML
**Data Lakes**
The volume, velocity and variety of data has been ever increasing since the advent of the internet. The problem many enterprises face is managing this “big data” and trying to make sense out of it to yield the most desirable outcome. Siloes in enterprises, continuous ingestion of data in numerous formats, and the ever-changing technology landscape make it difficult to collect, store, share, analyze, and visualize data. The question is: How do you build that robust data pipeline that connects to the data store and provides data scientists and engineers a platform to gain insights into your data?

Many organizations are starting to invest in a centralized data store and coming to understand the benefits of using Amazon Simple Storage Service (S3) as their data lake. Building a data pipeline from the data lake involves a data discovery process that includes identifying the data format and schema, cataloging the data, understanding the metadata to build table definitions, and providing the ability to query the data.

This notebook post walks you through all the steps required to build the data pipeline—from creating a crawler for your data in Amazon S3 and using AWS Glue for data discovery and cataloging, to using Amazon Athena to store and retrieve table metadata, and Amazon SageMaker to query Athena tables.

**Machine Learning**
In today’s connected world, it’s important for companies to monitor social media channels to protect their brand and customer relationships. Companies try to learn about their customers, products, and services through social media, emails, and other communications. Machine learning (ML) models can help address some of these needs. However, the process to build and train your own model can be complicated and slow. The Amazon machine learning platform provides pre-trained models that can be accessed within Amazon SageMaker using a Jupyter Notebook. Amazon SageMaker is a fully managed end to end ML platform with modular design, but we will use only a hosted notebook instance for this example. Amazon Comprehend is a natural language processing (NLP) service that uses machine learning to find insights and relationships in text.

Source: https://aws.amazon.com/blogs/machine-learning/run-sql-queries-from-your-sagemaker-notebooks-using-amazon-athena/


### 0. Prerequisites

#### 1. Crawl & Catalog Open Dataset
Create a crawler for Amazon Reviews public dataset.


1. Open the AWS Glue console and select Crawlers from the left navigation pane. Add a new crawler and give it a name. 
1. Next, in the Include Path field specify the S3 path to your Parquet data that we outputted in the previous section. In the Exclude Patterns field add the following two patterns: _metadata & _common_metadata
    - Include path	s3://amazon-reviews-pds/parquet/
    - Exclude patterns	metadata, _common_metadata
1. Choose Next, and when asked to add another data store leave it on No and choose Next.  
1. Select an existing IAM role or create a new one and choose Next. 
1. You can leave the Frequency as Run on demand because we’ll run this only once. 
1. On the next page in the console you are asked to select or create a Database to keep tables created by the crawler. Go ahead and create one or use the default. Additionally, you can give your table name a Prefix since table names are generated for you to avoid possible name collisions.  
1. Finally, click Next, and then Finish. 
1. Put a check mark next to your crawler in the main crawler window and choose Run Crawler. 
1. After the crawler completes its task you’ll have a new table in your Data Catalog. It should look something like this:


As data source provide following URL:

`s3://amazon-reviews-pds/tsv`

For more information on AWS Reviews public open dataset 
Refer to : https://s3.amazonaws.com/amazon-reviews-pds/readme.html

The dataset is currently available in two file formats.

Tab separated value (TSV), a text format - s3://amazon-reviews-pds/tsv/
Parquet, an optimized columnar binary format - s3://amazon-reviews-pds/parquet/
To further improve query performance the Parquet dataset is partitioned (divided into subfolders) on S3 by product_category. This allows for queries using a WHERE clause on product_category to only read data specific to that category.

#### 2. Permissions
**Important** Make sure, the IAM role attached to sagemaker notebook has 'Athena + Comprehend full access'

### 0. Getting started

This notebook uses Pyhton3 kernel

<TBD>

We will write a script that:

1. TBD

Some concepts:

- Spark provides a unified platform for writing big data applications, ranging from simple data loading and SQL queries to machine learning and streaming computation over the same engine and with a consistent set of APIs.
- Spark handles loading data from Amazon S3. 
- You control your Spark Application through a driver process called the SparkSession.
- A Spark DataFrame is the most common Structured API and simply represents a table of data with rows and columns. (Not to be confused with R and Python DataFrames. Those (with some exceptions) exist on one machine rather than multiple machines)
- Schema is the list that defines the columns and types within those columns.



### 1. Install PyAthena

Amazon SageMaker provides several kernels for Jupyter including support for Python 2 and 3, MXNet, TensorFlow, and PySpark. Choose Python as the kernel for this exercise as it comes with the Pandas library built in. Within the notebook, execute the following commands to install the Athena JDBC driver. PyAthena is a Python DB API 2.0 (PEP 249) compliant client for the Amazon Athena JDBC driver.

In [ ]:
import sys
!{sys.executable} -m pip install PyAthena

### 2. Query dataset on S3 using Athena

You can use the JDBC connection to connect to Athena and populate the Pandas data frames. For data scientists, working with data is typically divided into multiple stages: munging and cleaning data, analyzing / modeling it, then organizing the results of the analysis into a form suitable for plotting or tabular display. Pandas is the ideal tool for all of these tasks.

In [ ]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://hbaawsops-lz/athenaqresults/',
               region_name='us-east-1')


In [ ]:
df_bigdata = pd.read_sql("SELECT * FROM amazon_reviews.parquet limit 100000;", conn)

In [ ]:
df_bigdata.dtypes

In [ ]:
df_bigdata.shape

In [ ]:
df_bigdata.sample(3)

In [ ]:
import sys
!{sys.executable} -m pip install pixiedust

In [ ]:
from pixiedust.display import *


In [ ]:

display(df_bigdata.head(5))

### 3. Perform exploratory data analysis
You can perform exploratory data analysis on the result dataset to answer important business questions. 
Let's find out the products with the highest star rating

In [ ]:
print("Products that generated highest star rating in the descending order: ")

df_topreviews = df_bigdata.groupby( by = 'product_title')['star_rating'].mean().sort_values(ascending=False)

In [ ]:
df_topreviews.head(5)

### 4. Save the data to Amazon S3
Finally, you can save the results in a comma separated file (CSV) to Amazon S3. This allows you to reuse the  analysis results from Athena to build a business intelligence (BI) dashboard or to build other relevant machine learning models using Amazon SageMaker native algorithms.

In [ ]:
# Function to upload to S3
from io import StringIO
import boto3

def write_pd_s3_csv(df, bucket, filepath):
    csv_buffer = StringIO()
    df_bigdata.to_csv(csv_buffer)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, filepath).put(Body=csv_buffer.getvalue())
    print("The data is successfully written to S3 path:", bucket+"/"+filepath)


In [ ]:
# Write to S3
s3_bucket =  'prj-ml-lz'  
file_path = 'architecting-ml-aws/mod04-bigdata-ml/amazonreviewdata-highest-ranked-products.csv'
write_pd_s3_csv(df_bigdata[['marketplace','product_id','product_title','star_rating','verified_purchase','product_category']], s3_bucket, file_path)

### Comprehend

Amazon Comprehend takes your unstructured data such as social media posts, emails, webpages, documents, and transcriptions as input. Then it analyzes the input using the power of NLP algorithms to extract key phrases, entities, and sentiments automatically. It can also detect language of the input data and find relevant groupings of the data using topic modeling algorithms. 

**Important** Make sure, the IAM role attached to sagemaker notebook has 'Comprehend full access'

In [ ]:
import boto3 
comprehend = boto3.client('comprehend', region_name='us-east-1')

In [ ]:
df_reviews = pd.read_sql("SELECT marketplace, \
        product_title, \
        review_body, \
        year, \
        product_category \
    FROM amazon_reviews.parquet \
    WHERE star_rating < 3  limit 500;",
                         conn)


In [ ]:
df_reviews.dtypes

In [ ]:
df_reviews.sample(10)

### Athena
Alternatively, run this query on Athena

`SELECT "review_body" FROM "amzreviews"."reviews" WHERE star_rating < 3  limit 10;`


The item no 5 output is as follows:

`Worst product that I've ever purchased from Amazon. Completely useless. Wish I had read the previous reviews before making my purchase. The mesh bag would not hold soap and the item went directly into the trash.`

Let's see how Comprehend estimates tje

In [ ]:
df_lowest_reviews = pd.read_sql("SELECT review_body FROM amazon_reviews.parquet WHERE star_rating < 3  limit 10;", conn)

In [ ]:
df_lowest_reviews.head(10)

In [ ]:
sample_review_body="Worst product that I've ever purchased from Amazon. Completely useless. Wish I had read the previous reviews before making my purchase. The mesh bag would not hold soap and the item went directly into the trash."   

# Key phrases
phrases = comprehend.detect_key_phrases(Text=sample_review_body, LanguageCode='en')

# Entities
entities = comprehend.detect_entities(Text=sample_review_body, LanguageCode='en')

#Sentiments
sentiments = comprehend.detect_sentiment(Text=sample_review_body, LanguageCode='en')


# Print the phrases:
print('------- phrases ---------')
for i in range(0, len(phrases['KeyPhrases'])):
    print((phrases['KeyPhrases'][i]['Text']))
    

# Print the entities with entitity type:
print('------- entity : entity type ---------')
for i in range(0, len(entities['Entities'])):
    print(entities['Entities'][i]['Text'] + ' : ' + entities['Entities'][i]['Type'] )
    
# Print the sentiment:
print('------- sentiment ---------')
print(sentiments['Sentiment'])

### 8. Congratulations
Congratulations! You've Finished this lab. 


### Putting it together
Congratulations! You've Finished this lab. 

You have done data engineering tasks using Big Data Analytics, beyond the limitations of your notebook. 

- For more information, refer to AWS Blog [How to scale sentiment analysis using Amazon Comprehend, AWS Glue and Amazon Athena](https://aws.amazon.com/blogs/machine-learning/how-to-scale-sentiment-analysis-using-amazon-comprehend-aws-glue-and-amazon-athena/)

Quote: 

>>
At this point, you should be able to explore and visualize the data in whichever way you like. What should be clear from this lab is the way in which we constructed a data processing workflow using serverless technologies to clean, transform, enrich, query, and visualize large amounts of data quickly. Additionally, the method in which we were able to enrich our large dataset with Amazon Comprehend, the AWS NLP service, directly in our ETL job demonstrates a powerful pattern. This pattern can be applied to any AWS service, as well as other services, provided that the appropriate SDK is used. Another useful pattern I’ve learned by putting this pipeline together is that Amazon Athena comes in really handy when you need to inspect data because it is persisted to Amazon S3 at different stages of the pipeline." 

- For more information on serverless data lake concepts, refer to the other workshop prepared by the author [Serverless Data Lake Workshop](https://serverless-data-lake-immersionday.workshop.aws/en)